# Race Dashboard



This document describes the requirements and design decisions we will adopt for the Race Dashboard for the Sanofi Asset Efficiency challenge.  This document is intended to provide a detailed explanation of what is to be included on the dashboard, required data sources, and assumptions being made in the design.  It will also show the steps for getting the data into the necessary format.

## Scope/objective

The objective is to be able to display the documented metrics and categories as best suited on a Dashboard presentation for Sanofi staff to be able to access.


## Metrics / categories
The metrics have been mapped into sectors to mimic different sectors of a race track.  The metrics are:
 
 
 
•     Race = 8 Laps = 8 Months   
•     Lap = Monthly Progress   
<br>
<br>

•     **Sector 1 = OEE Improvement   
•     Sector 2 = OEE Variability Improvement   
•     Sector 3 = Stoppage Reduction   
•     Sector 4 = Changeover Improvement**   
• _Sector 5 = Most effective OEE application   
• Sector 6 = Best Innovation   
• Sector 7 = Most consistent OEE improvement progress   
• Sector 8 = Collaboration   
• Sector 9 = Team Spirit_

 
 


| Change log |
|:----------:|    


| Date | Initials | Comments |
|------|:---------|:---------|
| 2021-06-23 | MC | in leaderboard, replace NaN values in laptime calc with the max laptime for that lap
| 2021-06-23 | MC | use race review dates for grouping data, rather than calendar months
| 2021-06-24 |JB | missing OEE_Diff figures should default to 'OEE %' - OEE start point, not just 'OEE %' value
| 2021-06-24 |JB | for sector one, multiply sum of OEE_Diff by -1.  (was doing this in Tableau calcs but too late for lap_time calc).
|2021-06-25 | MC | in leaderboard, change prev_race_time calc to include all but last 2 cols, to handle new race cols as they arrive
|2021-06-28 | MC | correct dates in nominations spreadsheet, and merge on 1 row with a date within the review period
|2021-06-29 | MC | corrected nominations spreadsheet for Lisieux IWK - should be TR200 Packaging
|2021-06-30 | MC | populate missing OEE % values before calculating rolling_std using the OEE % column
|2021-06-30 | MC | remove start_changeover_calc merge as it's not being used any more, and was dropping rows
|2021-07-06 | MC | populate missing OEE % values with OEE from previous week for that site.

In [366]:
import pandas as pd
import numpy as np
import datetime


# Viz libs
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

# display options
# pd.options.display.float_format = "{:.2f}".format


## Read file and cleanse

- OEE.xlsx should contain the OEE latest data from QS by week   
- QSDashboard.xlsx has a list of plants/sites taking part, with original target OEE  
- UnplannedTechLoss.xlxs should contain the latest data from QS (my unplanned chart by line/week)   
- changeover.xlsx should contain the changeover information in QlikSense

#### Cleaning required:   
- OEE % needs converting to numeric, coerce the nulls to nan values
- I think W53-2020 data is bobbings - causes dup indexes for date 2021-01-10 and don't need 2020 data anyway...so dropping it

In [384]:
# this dir should point at where ever your sanofi Mclaren directory is syncing
dir = "C:/Users/mark_/Sanofi/Sanofi x McLaren sharing - General/Race Dashboard data/"

# this is a local copy of sanofi share for when I don't trust Microsoft syncing!
# dir = "C:/Users/mark_/Documents/McLaren2021/Sanofi/Race Dashboard data/"

# write out the files where ever you like.  As long as same file is used within Tableau.
output_dir = "C:/Users/mark_/Documents/McLaren2021/Sanofi/Race Dashboard data/"

# dir = 'C:/Users/james.blood/Documents/McLarenSanofi/McLarenSanofi/data/'

file = (dir + 'OEE.xlsx')
df_weekly = pd.read_excel(file)
file = (dir + 'QSDashboard.xlsx')
df_dash = pd.read_excel(file)
file = (dir + 'Unplanned_tech_loss.xlsx')
df_techloss = pd.read_excel(file)
file = (dir + 'changeover.xlsx')
df_changeover = pd.read_excel(file)
df_weekly = df_weekly.loc[df_weekly['Week'].str.contains('2021')]
df_techloss = df_techloss.loc[df_techloss['Week'].str.contains('2021')]
df_weekly['OEE %'] = pd.to_numeric(df_weekly['OEE %'], errors='coerce')
df_techloss.rename(columns={'Unplanned losses - %OEE':'Unplanned_tech_loss'}, inplace=True)
df_techloss['Unplanned_tech_loss'] = pd.to_numeric(df_techloss['Unplanned_tech_loss'], errors='coerce')
df_changeover.rename(columns={'Change over losses - %OEE':'Changeover'}, inplace=True)
df_changeover['Changeover'] = pd.to_numeric(df_changeover['Changeover'], errors='coerce')
# don't use their progress figure as it's a static val
# df_dash.rename(columns={'⇗ OEE% progress':'OEE% progress'}, inplace=True)

create a datetime from the week number

In [385]:
df_weekly['WeekOfYear'] = pd.to_numeric(df_weekly['Week'].str[1:3])
df_weekly['Year'] = pd.to_numeric(df_weekly['Week'].str[4:])
dates = df_weekly.Year*100+df_weekly.WeekOfYear
df_weekly['Date'] = pd.to_datetime(dates.astype(str) + '0', format='%Y%W%w')
# df_weekly.drop(columns=['Year','WeekOfYear'], inplace=True)
df_weekly.head()

,Week,Line,OEE %,WeekOfYear,Year,Date
11,W01-2021,C2 Packaging Line,0.168970,1,2021,2021-01-10
12,W01-2021,C9 Packaging Line,NaN,1,2021,2021-01-10
13,W01-2021,GAMMA1,0.406686,1,2021,2021-01-10
14,W01-2021,IMA C80/2,0.510044,1,2021,2021-01-10
15,W01-2021,L18 Packaging Line,0.173736,1,2021,2021-01-10


In [386]:
#merge the 2 dataframes to get the start OEE
df_weekly = df_weekly.merge(df_dash[['Plant','Line', 'OEE  Start point','OEE% Target (2022)']],on='Line')

In [387]:
df_weekly = df_weekly.merge(df_techloss[['Line', 'Week', 'Unplanned_tech_loss']],on=['Line','Week'])

In [388]:
df_weekly = df_weekly.merge(df_changeover[['Line','Week','Changeover']])

#### Start Changeover

*not sure this is needed if we are using changeover_diff, which will naturally reward increases / decreases in changeover??*

Start changeover value isn't provided, so going to calc our own start point using the average changeover for each site in 2021 up to April 2021.  This needs to be done before we drop the early 2021 rows.

This is then merged into the df_weekly dataframe as a loose join.

In [389]:
# start_changeover_calc = df_weekly[['Plant','Line','Changeover']][df_weekly['Date'] < '2021-04-30'].groupby(['Plant','Line']).mean().reset_index()
# start_changeover_calc.rename(columns={'Changeover':'start_changeover'}, inplace=True)
# df_weekly = df_weekly.merge(start_changeover_calc[['Line','start_changeover']])

#### Dates for the Asset Challenge

Start Date is going to be fixed as 2021-04-01. Remove all the rows from df_weekly before this date

Review Date will move and act as a cutoff for each Race meeting

In [390]:
start_date = '2021-04-01'
df_weekly = df_weekly[df_weekly['Date'] > start_date].sort_values('Date')

# this is needed to prevent including additional values in the race time.
review_date = '2021-07-08'
# df_weekly = df_weekly[df_weekly['Date'] < end_date].sort_values('Date')

### PCT_CHANGE
Using pct_change python function with periods=4, giving a 4 week (4 previous rows) rolling pct_change figure
- I believe we are doing this rolling average calculation within Tableau at the moment, so this isn't being used here

- Not sure whether this is required any more - removing for all categories (2021-06-30)

In [391]:
# df_weekly.sort_values(['Line','Date'], inplace = True)
# df_weekly['OEE_pct_chg'] = (df_weekly.groupby('Line')['OEE %']
#                                    .apply(pd.Series.pct_change, periods=4))
# df_weekly['techloss_pct_chg'] = (df_weekly.groupby('Line')['Unplanned_tech_loss']
#                                    .apply(pd.Series.pct_change, periods=4))
# df_weekly['Changeover_pct_chg'] = (df_weekly.groupby('Line')['Changeover']
#                                    .apply(pd.Series.pct_change, periods=4))
# df_weekly.head()

#### Populate missing OEE %
2021-06-07 - change of plan! Populating with min OEE value does not work when some Lines record 0% OEE.  Missing values can be just delayed data collection which sorts itself later.  So, until proper data arrives:

- Use the previous week's value (ffill) for that Line.  This will still leave NaN if they existed in the first row(s).  
- populate the remaining NaN with 'OEE  start point'
- ~~Find the weekly min/max OEE % from any site~~   
- ~~Merge those columns into df_weekly~~   
- ~~fill any NaN with the min OEE we calc'd for that week~~    


In [392]:
# df_weekly_minmax = (df_weekly.assign(Data_Value=df_weekly['OEE %'].abs())
#        .groupby(pd.Grouper(key='Date',freq='W'))['OEE %'].agg([('Min' , 'min'), ('Max', 'max')])
#        .add_prefix('Week'))
# df_weekly_minmax.reset_index(inplace=True)
# df_weekly = df_weekly.merge(df_weekly_minmax[['Date','WeekMin','WeekMax']])
# df_weekly['OEE %'].fillna(df_weekly.WeekMin, inplace=True)
# df_weekly.drop(columns=['WeekMin','WeekMax'], inplace=True)
df_weekly['OEE %'] = df_weekly.sort_values(['Line','Date']).groupby(['Line'])['OEE %'].ffill()
df_weekly['OEE %'] = df_weekly.sort_values(['Line','Date']).groupby(['Line'])['OEE %'].fillna(df_weekly['OEE  Start point'])
# df_weekly[df_weekly['OEE %'].isna()]
df_weekly[df_weekly.Line.str.contains('M18')]

,Week,Line,OEE %,WeekOfYear,Year,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,Changeover
174,W13-2021,M18 Filling,0.046124,13,2021,2021-04-04,Frankfurt,0.443522,0.65,0.114818,0.789781
175,W14-2021,M18 Filling,0.463021,14,2021,2021-04-11,Frankfurt,0.443522,0.65,0.240339,0.095404
176,W15-2021,M18 Filling,0.386723,15,2021,2021-04-18,Frankfurt,0.443522,0.65,0.069673,0.390417
177,W16-2021,M18 Filling,0.352420,16,2021,2021-04-25,Frankfurt,0.443522,0.65,0.485988,0.033241
178,W17-2021,M18 Filling,0.530336,17,2021,2021-05-02,Frankfurt,0.443522,0.65,0.311511,0.029190
179,W18-2021,M18 Filling,0.562696,18,2021,2021-05-09,Frankfurt,0.443522,0.65,0.227898,0.037331
180,W19-2021,M18 Filling,0.698908,19,2021,2021-05-16,Frankfurt,0.443522,0.65,0.108069,0.000000
181,W20-2021,M18 Filling,0.000000,20,2021,2021-05-23,Frankfurt,0.443522,0.65,0.000000,1.000000
182,W21-2021,M18 Filling,0.191951,21,2021,2021-05-30,Frankfurt,0.443522,0.65,0.080336,0.626590
183,W22-2021,M18 Filling,0.537044,22,2021,2021-06-06,Frankfurt,0.443522,0.65,0.216673,0.038185


## Standard Deviation
Calculate std_dev and mean on a 4 week rolling basis

Standard deviation is the square root of the variance, so no need to calculate both and have left var out

In [393]:
df_weekly['rolling_std'] = df_weekly.groupby('Line')['OEE %'].apply(lambda x : x.rolling(4,1).agg(np.std))
df_weekly.head(5)

,Week,Line,OEE %,WeekOfYear,Year,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,Changeover,rolling_std
12,W13-2021,C2 Packaging Line,0.458414,13,2021,2021-04-04,Maisons-Alfort,0.397503,0.47,0.465009,0.044140,NaN
228,W13-2021,M22 Filling,0.649336,13,2021,2021-04-04,Frankfurt,0.530068,0.65,0.234635,0.035164,NaN
255,W13-2021,SUPPO Packaging Line,0.432432,13,2021,2021-04-04,Lisieux,0.353021,0.53,0.148267,0.178806,NaN
174,W13-2021,M18 Filling,0.046124,13,2021,2021-04-04,Frankfurt,0.443522,0.65,0.114818,0.789781,NaN
39,W13-2021,C9 Packaging Line,0.371845,13,2021,2021-04-04,Maisons-Alfort,0.528518,0.53,0.229297,0.081871,NaN


In [394]:
file = (dir + 'Nominations Category Scoring.xlsx')
df_nom_sectors = pd.read_excel(file, sheet_name='Nomination scoring', usecols="A:H", parse_dates=['Date'])

In [395]:
df_nom_sectors['Date'] = pd.Series(df_nom_sectors['Date']).fillna(method='ffill')
df_nom_sectors = df_nom_sectors.fillna(0)

df_weekly = df_weekly.merge(df_nom_sectors[['Line','Plant','Date','Best Solution','Best Innovation','Improvement Iterations','Lessons and Sharing','Team Contribution and Spirit']],how='outer', on=['Date','Plant','Line'])
df_weekly

,Week,Line,OEE %,WeekOfYear,Year,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,Changeover,rolling_std,Best Solution,Best Innovation,Improvement Iterations,Lessons and Sharing,Team Contribution and Spirit
0,W13-2021,C2 Packaging Line,0.458414,13.0,2021.0,2021-04-04,Maisons-Alfort,0.397503,0.47,0.465009,0.044140,NaN,NaN,NaN,NaN,NaN,NaN
1,W13-2021,M22 Filling,0.649336,13.0,2021.0,2021-04-04,Frankfurt,0.530068,0.65,0.234635,0.035164,NaN,NaN,NaN,NaN,NaN,NaN
2,W13-2021,SUPPO Packaging Line,0.432432,13.0,2021.0,2021-04-04,Lisieux,0.353021,0.53,0.148267,0.178806,NaN,NaN,NaN,NaN,NaN,NaN
3,W13-2021,M18 Filling,0.046124,13.0,2021.0,2021-04-04,Frankfurt,0.443522,0.65,0.114818,0.789781,NaN,NaN,NaN,NaN,NaN,NaN
4,W13-2021,C9 Packaging Line,0.371845,13.0,2021.0,2021-04-04,Maisons-Alfort,0.528518,0.53,0.229297,0.081871,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,NaN,AL5 Packaging 1,NaN,NaN,NaN,2021-09-12,Frankfurt,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
243,NaN,AL6,NaN,NaN,NaN,2021-09-12,Frankfurt,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
244,NaN,M18 Filling,NaN,NaN,NaN,2021-09-12,Frankfurt,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
245,NaN,M21 Filling,NaN,NaN,NaN,2021-09-12,Frankfurt,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0


## Create review dates
create a review_date column for grouping the data later, so we only get the data we're interested in for each review

In [396]:
def thurs_of_weekbefore(year, week):
    return datetime.date.fromisocalendar(year, week-1, 4)  # (year, week before (w-1), thursday)

review_weeks = [16, 20, 24, 28, 34, 38, 42, 47]
review_dates = []

for i in review_weeks:
    if i > 0:
        review_dates.append((thurs_of_weekbefore(2021,i)))

df_review_dates = pd.DataFrame(review_dates)
df_review_dates.rename(columns={0:'Review_Date'}, inplace=True)
df_review_dates['Review_Date'] = pd.to_datetime(df_review_dates.Review_Date)

# df_review_dates.info()
df_weekly = pd.merge_asof(df_weekly.sort_values('Date'), df_review_dates, left_on='Date', right_on='Review_Date', allow_exact_matches=True, direction='forward')


#### We need the diff between the weekly OEE % figures, and the Weekly Changeover figures
Need something to calculate the OEE Progress and Changeover, otherwise we will have problems when we group and sum values later
- Create OEE_diff with OEE % from groupby of each Line, Week (only 1 row per week, so 'mean' will yield the same)   
- Find the diff between the rows in OEE_Diff for each Line   
- fillNA (first row for each Line) with OEE Start point - should only be needed on the first row for each Line   

repeat same logic for Changeover - there will be more NaN as start_changeover wasn't provided for all.  We populate this later

In [397]:
# this was calculating the wrong Diff - the first row of each site was looking at the previous site for all but the 1st calc
# needed to sort by Line and Date first 

# OEE_Diff = df_weekly.groupby(['Line',pd.Grouper(key='Date',freq='W')])['OEE %'].mean().reset_index()
# OEE_Diff["OEE_Diff"] = OEE_Diff["OEE %"].diff()
# df_weekly = df_weekly.merge(OEE_Diff[["Line","Date","OEE_Diff"]], on=(["Line","Date"]))

# df_weekly['OEE_Diff'].fillna(df_weekly['OEE %'] - df_weekly['OEE  Start point'], inplace=True)
# df_weekly[["Line","Date","OEE %","OEE_Diff"]].head(50).sort_values(by=['Line', 'Date'])

after merging with nominations spreadsheet we will have future dated rows that get in the way of calcs later - get rid of them using the review_date set earlier

In [399]:
df_weekly = df_weekly[df_weekly['Review_Date'] <= review_date]

In [400]:
OEE_Diff = df_weekly.sort_values(by=['Line', 'Date'])[['Line','Date','OEE %','OEE  Start point']]
OEE_Diff['OEE_Diff'] = OEE_Diff.groupby('Line')['OEE %'].diff().fillna(df_weekly['OEE %'] - df_weekly['OEE  Start point'])
df_weekly = df_weekly.merge(OEE_Diff[["Line","Date","OEE_Diff"]], on=(["Line","Date"]))
df_weekly[["Line","Date","OEE %","OEE_Diff"]].head(50).sort_values(by=['Line', 'Date'])

,Line,Date,OEE %,OEE_Diff
12,AL5 Packaging 1,2021-04-04,0.479693,0.000000
25,AL5 Packaging 1,2021-04-11,0.479693,0.000000
30,AL5 Packaging 1,2021-04-18,0.449745,-0.029948
3,AL6,2021-04-04,0.367897,0.035240
16,AL6,2021-04-11,0.360681,-0.007216
31,AL6,2021-04-18,0.334140,-0.026541
45,AL6,2021-04-25,0.309545,-0.024595
0,C2 Packaging Line,2021-04-04,0.458414,0.060911
19,C2 Packaging Line,2021-04-11,0.530707,0.072293
38,C2 Packaging Line,2021-04-18,0.410676,-0.120032


In [401]:
# Changeover_Diff = df_weekly.sort_values(by=['Line', 'Date'])[['Line','Date','Changeover','start_changeover']]
Changeover_Diff = df_weekly.sort_values(by=['Line', 'Date'])[['Line','Date','Changeover']]
# Changeover_Diff['Changeover_Diff'] = Changeover_Diff.groupby('Line')['Changeover'].diff().fillna(df_weekly['start_changeover'] - df_weekly['Changeover'])
Changeover_Diff['Changeover_Diff'] = Changeover_Diff.groupby('Line')['Changeover'].diff().fillna(0)
df_weekly = df_weekly.merge(Changeover_Diff[["Line","Date","Changeover_Diff"]], on=(["Line","Date"]))

In [402]:
Changeover_Diff[Changeover_Diff.Line.str.contains('AL5')]

,Line,Date,Changeover,Changeover_Diff
12,AL5 Packaging 1,2021-04-04,NaN,0.000000
25,AL5 Packaging 1,2021-04-11,NaN,0.000000
30,AL5 Packaging 1,2021-04-18,0.087039,0.000000
51,AL5 Packaging 1,2021-04-25,0.122727,0.035688
54,AL5 Packaging 1,2021-05-02,0.063261,-0.059466
77,AL5 Packaging 1,2021-05-09,0.089293,0.026032
91,AL5 Packaging 1,2021-05-16,0.060573,-0.028721
108,AL5 Packaging 1,2021-05-23,NaN,0.000000
114,AL5 Packaging 1,2021-05-30,NaN,0.000000
129,AL5 Packaging 1,2021-06-06,0.088889,0.000000


In [403]:
# Changeover_mean = df_weekly.sort_values(by=['Line', 'Date'])[['Line','Date','Changeover','start_changeover']]
Changeover_mean = df_weekly.sort_values(by=['Line', 'Date'])[['Line','Date','Changeover']]
Changeover_mean['Changeover_rolling_mean'] = Changeover_mean.groupby('Line')['Changeover'].apply(lambda x : x.rolling(4,1).mean())
df_weekly = df_weekly.merge(Changeover_mean[["Line","Date","Changeover_rolling_mean"]], on=(["Line","Date"]))

In [404]:
Changeover_mean[Changeover_mean.Line.str.contains('LINE')]

,Line,Date,Changeover,Changeover_rolling_mean
65,LINE 01 - UHLMANN 1880,2021-05-02,0.000000,0.000000
73,LINE 01 - UHLMANN 1880,2021-05-09,0.000000,0.000000
89,LINE 01 - UHLMANN 1880,2021-05-16,0.016561,0.005520
100,LINE 01 - UHLMANN 1880,2021-05-23,0.040157,0.014179
126,LINE 01 - UHLMANN 1880,2021-05-30,0.024481,0.020300
132,LINE 01 - UHLMANN 1880,2021-06-06,0.205835,0.071758
142,LINE 01 - UHLMANN 1880,2021-06-13,0.025332,0.073951
166,LINE 01 - UHLMANN 1880,2021-06-20,0.023659,0.069827
184,LINE 01 - UHLMANN 1880,2021-06-27,0.018125,0.068238
195,LINE 01 - UHLMANN 1880,2021-07-04,0.094504,0.040405


#### Populate missing Unplanned Tech Loss

- Create weekly min/max cols for Unplanned tech loss from any site   
- Merge those columns into df_weekly   
- fill any NaN rows with the max Unplanned_tech_loss found for that week   

**this might be flawed!!** 

In [405]:
df_weekly_minmax = (df_weekly.assign(Data_Value=df_weekly['Unplanned_tech_loss'].abs())
       .groupby(pd.Grouper(key='Date',freq='W'))['Unplanned_tech_loss'].agg([('Min' , 'min'), ('Max', 'max')])
       .add_prefix('WeekUTL'))
df_weekly_minmax.reset_index(inplace=True)
df_weekly = df_weekly.merge(df_weekly_minmax[['Date','WeekUTLMin','WeekUTLMax']])
df_weekly['Unplanned_tech_loss'].fillna(df_weekly.WeekUTLMax, inplace=True)
df_weekly.drop(columns=['WeekUTLMin','WeekUTLMax'], inplace=True)
df_weekly

,Week,Line,OEE %,WeekOfYear,Year,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,...,rolling_std,Best Solution,Best Innovation,Improvement Iterations,Lessons and Sharing,Team Contribution and Spirit,Review_Date,OEE_Diff,Changeover_Diff,Changeover_rolling_mean
0,W13-2021,C2 Packaging Line,0.458414,13.0,2021.0,2021-04-04,Maisons-Alfort,0.397503,0.470,0.465009,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.060911,0.000000,0.044140
1,W13-2021,L18 Packaging Line,0.465923,13.0,2021.0,2021-04-04,Tours,0.377683,0.547,0.208273,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.088240,0.000000,0.134271
2,W13-2021,IMA C80/2,0.581304,13.0,2021.0,2021-04-04,SCOPPITO,0.451031,0.580,0.224561,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.130272,0.000000,0.070836
3,W13-2021,AL6,0.367897,13.0,2021.0,2021-04-04,Frankfurt,0.332657,0.450,0.342851,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.035240,0.000000,0.184958
4,W13-2021,TR200 Packaging Line,0.596432,13.0,2021.0,2021-04-04,Lisieux,0.483505,0.650,0.163573,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.112928,0.000000,0.075189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,W26-2021,GAMMA1,0.491885,26.0,2021.0,2021-07-04,SCOPPITO,0.418683,0.570,0.143339,...,0.055086,0.0,0.0,0.0,0.0,0.0,2021-07-08,-0.111172,0.000000,0.219199
198,W26-2021,C9 Packaging Line,0.406623,26.0,2021.0,2021-07-04,Maisons-Alfort,0.528518,0.530,0.452948,...,0.081720,0.0,0.0,0.0,0.0,0.0,2021-07-08,-0.082118,-0.057612,0.064467
199,W26-2021,L18 Packaging Line,0.408843,26.0,2021.0,2021-07-04,Tours,0.377683,0.547,0.184455,...,0.054692,0.0,0.0,0.0,0.0,0.0,2021-07-08,-0.084213,0.036575,0.182758
200,W26-2021,L25 Packaging Line,0.381273,26.0,2021.0,2021-07-04,Tours,0.351564,0.478,0.192098,...,0.019916,0.0,0.0,0.0,0.0,0.0,2021-07-08,0.036620,-0.031079,0.252406


#### Populate missing Changeover 

In [406]:
df_weekly_minmax = (df_weekly.assign(Data_Value=df_weekly['Changeover'].abs())
       .groupby(pd.Grouper(key='Date',freq='W'))['Changeover'].agg([('Min' , 'min'), ('Max', 'max')])
       .add_prefix('WeekChangeover'))
df_weekly_minmax.reset_index(inplace=True)
df_weekly = df_weekly.merge(df_weekly_minmax[['Date','WeekChangeoverMin','WeekChangeoverMax']])
df_weekly['Changeover'].fillna(df_weekly.WeekChangeoverMax, inplace=True)
df_weekly.drop(columns=['WeekChangeoverMin','WeekChangeoverMax'], inplace=True)
df_weekly

,Week,Line,OEE %,WeekOfYear,Year,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,...,rolling_std,Best Solution,Best Innovation,Improvement Iterations,Lessons and Sharing,Team Contribution and Spirit,Review_Date,OEE_Diff,Changeover_Diff,Changeover_rolling_mean
0,W13-2021,C2 Packaging Line,0.458414,13.0,2021.0,2021-04-04,Maisons-Alfort,0.397503,0.470,0.465009,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.060911,0.000000,0.044140
1,W13-2021,L18 Packaging Line,0.465923,13.0,2021.0,2021-04-04,Tours,0.377683,0.547,0.208273,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.088240,0.000000,0.134271
2,W13-2021,IMA C80/2,0.581304,13.0,2021.0,2021-04-04,SCOPPITO,0.451031,0.580,0.224561,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.130272,0.000000,0.070836
3,W13-2021,AL6,0.367897,13.0,2021.0,2021-04-04,Frankfurt,0.332657,0.450,0.342851,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.035240,0.000000,0.184958
4,W13-2021,TR200 Packaging Line,0.596432,13.0,2021.0,2021-04-04,Lisieux,0.483505,0.650,0.163573,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-15,0.112928,0.000000,0.075189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,W26-2021,GAMMA1,0.491885,26.0,2021.0,2021-07-04,SCOPPITO,0.418683,0.570,0.143339,...,0.055086,0.0,0.0,0.0,0.0,0.0,2021-07-08,-0.111172,0.000000,0.219199
198,W26-2021,C9 Packaging Line,0.406623,26.0,2021.0,2021-07-04,Maisons-Alfort,0.528518,0.530,0.452948,...,0.081720,0.0,0.0,0.0,0.0,0.0,2021-07-08,-0.082118,-0.057612,0.064467
199,W26-2021,L18 Packaging Line,0.408843,26.0,2021.0,2021-07-04,Tours,0.377683,0.547,0.184455,...,0.054692,0.0,0.0,0.0,0.0,0.0,2021-07-08,-0.084213,0.036575,0.182758
200,W26-2021,L25 Packaging Line,0.381273,26.0,2021.0,2021-07-04,Tours,0.351564,0.478,0.192098,...,0.019916,0.0,0.0,0.0,0.0,0.0,2021-07-08,0.036620,-0.031079,0.252406


#### Populate missing start_changeover (NaN)
If we've got this far and you still don't have a start_changeover, then you're a new site and can have this week's Changeover value

These rows still have NaN Changeover values

In [407]:
df_weekly[['Line','Date','Changeover']][df_weekly['Changeover'].isna()]

,Line,Date,Changeover


In [408]:
# df_weekly.start_changeover.fillna(df_weekly.Changeover, inplace=True)
df_weekly['OEE  Start point'].fillna(df_weekly['OEE %'], inplace=True)
df_weekly['OEE% Target (2022)'].fillna(0.65, inplace=True)

### Calculating Sector times


The lap time is a sum of the calculated sector scores + the pole position time from the F1 race data (eg for Paul Ricard it was 88 secs):



Sector [1-4] calculations   
**sector 1**
How much has your OEE increased / decreased?  Sum difference between each week and multiply total by -1.  This provides a negative figure to subtract from your laptime, so that larger OEE increase is rewarded with a bigger reduction in laptime

df_weekly['sector_1'] = df_weekly['OEE_Diff'].mul(-1)

OEE_Diff calculation
- Sort values by Line and Date
- Find the difference between each weekly OEE figure
- ~~Fill NaN values from missing OEE figures with the weekly OEE minus OEE Start Point for that site~~   
- any missing OEE figures will already be populated with previous week's value or the start OEE value


**Sector 2** 
How big was your rolling std deviation this period, over the previous 4 weeks std dev?  

df_weekly['sector_2'] = df_weekly['rolling_std']

rolling_std = rolling std deviation for past 4 weeks for each site


 
**Sector 3**
We want to reduce Unplanned tech loss (recorded as % of OEE) Unplanned tech loss is calculated within QlikSense but missing values sometimes.  Fill the missing values and then display the average Unplanned tech loss :

df_weekly['sector_3'] = df_weekly['Unplanned_tech_loss']


Populate missing unplanned tech loss:
- Create weekly min/max cols for Unplanned tech loss from any site
- Merge those columns into df_weekly
- fill any NaN unplanned tech loss rows with the max OEE calc'd for that week (bigger is worse)

 
**Sector 4**
We're trying to reduce changeover time (recorded as % of OEE).  
Start changeover value isn't provided, so calc our own start point for each Line using the average changeover in 2021 up to 30 April, 2021.
   
start_changeover_calc = df_weekly[['Plant','Line','Changeover']][df_weekly['Date'] < '2021-04-30'].groupby(['Plant','Line']).mean().reset_index()
start_changeover_calc.rename(columns={'Changeover':'start_changeover'}, inplace=True)
df_weekly = df_weekly.merge(start_changeover_calc[['Line','start_changeover']])



df_weekly['sector_4'] = df_weekly['Changeover_rolling_mean']

Changeover_mean = df_weekly.sort_values(by=['Line', 'Date'])[['Line', 'Date', 'Changeover', 'start_changeover']]   
Changeover_mean['Changeover_rolling_mean'] = Changeover_mean.groupby('Line')['Changeover'].apply(lambda x : x.rolling(4,1).mean())
df_weekly = df_weekly.merge(Changeover_mean[["Line","Date","Changeover_rolling_mean"]], on=(["Line","Date"]))



**Clean the sectors of NaN before summing them**   
Sometimes, when we haven't got enough information for pct_change calcs, we were getting no values coming through for the lap_time.  We should make sure there is a value in each of the sectors, otherwise there is an unfair advantage by not having data available.  Find all NaN values and replace with the mean for that column(sector)

**Sectors 5 - 9**   
These scores are taken from the Nomination process.  Read in the Nomination s/s, merge any values we find with df_weekly, replace all NaN (missing) values with 0, and reduce the scores we find to 10% of their original value.  This value is then subtracted from the lap_time - so the better you do in the nominations the more your lap_time gets reduced by.

 
**lap_time**   
df_weekly['lap_time'] = df_weekly[['sector_1','sector_2','sector_3','sector_4','sector_5','sector_6','sector_7','sector_8','sector_9']].sum(axis=1)

In [409]:
# df_weekly['sector_1'] = (df_weekly['WeekMax'] - df_weekly['OEE %'])
# df_weekly['sector_1'] = (df_weekly['OEE  Start point'] - df_weekly['OEE %'])
df_weekly['sector_1'] = df_weekly['OEE_Diff'].mul(-1)
df_weekly['sector_2'] = df_weekly['rolling_std']
df_weekly['sector_3'] = df_weekly['Unplanned_tech_loss']
# df_weekly['sector_4'] = (df_weekly['start_changeover'] - df_weekly['Changeover'])
# df_weekly['sector_4'] = (df_weekly['Changeover'] - df_weekly['start_changeover'])
df_weekly['sector_4'] = df_weekly['Changeover_rolling_mean']
# take 10% of the sector5-9 scores 
df_weekly[['sector_5','sector_6','sector_7','sector_8','sector_9']] = df_weekly[['Best Solution','Best Innovation','Improvement Iterations','Lessons and Sharing','Team Contribution and Spirit']] * -0.1
df_weekly[['sector_5','sector_6','sector_7','sector_8','sector_9']] = df_weekly[['sector_5','sector_6','sector_7','sector_8','sector_9']].fillna(0)

# we'll use these in the absence of values for a sector
df_weekly[['sector_1','sector_2','sector_3','sector_4']] = df_weekly[['sector_1','sector_2','sector_3','sector_4']].fillna(df_weekly[['sector_1','sector_2','sector_3','sector_4']].mean())

#this will sum and handle the NaN
df_weekly['lap_time'] = df_weekly[['sector_1','sector_2','sector_3','sector_4','sector_5','sector_6','sector_7','sector_8','sector_9']].sum(axis=1)

# now add the pole['Laptime'] from fastf1 to the lap_time adjustment we've created
# just use 88 secs rather than playing with timedeltas for now
# df_weekly['lap_time'] = pole['LapTime'] + pd.to_timedelta(df_weekly['lap_time'], unit='S')
# df_weekly['lap_time'] = 88 + df_weekly['lap_time']
df_weekly.groupby(['Line', pd.Grouper(key='Date', freq='W')])['lap_time'].sum()
# print (df_weekly['sector_1_time'] , df_weekly['sector_2_time'] , df_weekly['sector_3_time'], df_weekly['sector_4_time'])

Line                  Date      
AL5 Packaging 1       2021-04-04    0.708826
                      2021-04-11    0.540147
                      2021-04-18    0.449241
                      2021-04-25    0.134643
                      2021-05-02    0.593747
                                      ...   
TR200 Packaging Line  2021-06-06    0.162536
                      2021-06-13    0.704221
                      2021-06-20    0.635328
                      2021-06-27    0.297245
                      2021-07-04    0.832851
Name: lap_time, Length: 202, dtype: float64

#### Write out df_weekly to excel

In [410]:
df_weekly.to_excel(output_dir + "df_weekly_with_calcs.xlsx")

#### Monthly Calcs

Repeat the process for a df_monthly spreadsheet.  We will use this for calculating the Leader board  
group df_weekly by review_date so we can get the right data for each review meeting

In [411]:
# df_monthly = df_weekly.groupby([pd.Grouper(key='Date',freq='M'),'Line'])[['start_changeover','OEE  Start point','OEE %','Unplanned_tech_loss','Changeover','rolling_std','techloss_pct_chg','Changeover_pct_chg']].mean().reset_index()
# df_monthly = df_weekly.groupby([pd.Grouper(key='Date',freq='M'),'Line']).lap_time.sum().reset_index()
df_monthly = df_weekly.groupby(['Review_Date','Line']).lap_time.sum().reset_index()
# change the name of review_date to save renaming all references to Date later
df_monthly = df_monthly.rename(columns={'Review_Date':'Date'})
df_monthly['lap_time'] = df_monthly['lap_time'] + 88
df_monthly.head()

,Date,Line,lap_time
0,2021-04-15,AL5 Packaging 1,89.248972
1,2021-04-15,AL6,89.137919
2,2021-04-15,C2 Packaging Line,88.952755
3,2021-04-15,C9 Packaging Line,89.021344
4,2021-04-15,GAMMA1,89.153635


### Leader board table

In [412]:
# filter using the end_date to stop picking up future dated nomination rows of zero I created when joining the s/s
# pivot = df_monthly[df_monthly['Date'] < end_date].pivot(index='Line', columns='Date', values='lap_time')
pivot = df_monthly.pivot(index='Line', columns='Date', values='lap_time')
pivot.reset_index(inplace=True)
# pivot creates NaN for rows with no monthly data for each race review data
# populate each NaN value with the max for that column - so they get the max laptime for that race
# we can search for cols [1:] and find all cols after Date and Line
pivot.iloc[:,1:] = pivot.iloc[:,1:].fillna(pivot.iloc[:,1:].max())

# sum all the columns to get a race_time
pivot['race_time'] = pivot.sum(axis=1)
# sum all but the last 2 cols (this lap and the race_time) to calc prev_race_time

# pivot['prev_race_time'] = pivot[pivot.columns[2]] + pivot[pivot.columns[3]]
pivot['prev_race_time'] = pivot.iloc[:,1:-2].sum(axis=1)

pivot = pivot.merge(df_dash[['Plant','Line']], on='Line')
pivot.sort_values('race_time', inplace=True)
pivot['position'] = np.arange(1,len(pivot) + 1)
pivot['gap_to_leader'] = pivot['race_time'] - pivot['race_time'].iloc[0]
pivot.sort_values('prev_race_time', inplace=True)
pivot['prev_position'] = np.arange(1,len(pivot) + 1)
pivot['Gain/Loss'] = pivot.prev_position - pivot.position
pivot.sort_values('race_time', inplace=True)
pivot['interval'] = pivot.race_time.diff()
pivot = pivot.merge(df_dash[['Line','OEE  Start point', '⇗ OEE% progress', 'OEE% Target (2022)']], on='Line')
pivot

,Line,2021-04-15 00:00:00,2021-05-13 00:00:00,2021-06-10 00:00:00,2021-07-08 00:00:00,race_time,prev_race_time,Plant,position,gap_to_leader,prev_position,Gain/Loss,interval,OEE Start point,⇗ OEE% progress,OEE% Target (2022)
0,GAMMA1,89.153635,69.325767,68.226756,89.979194,316.685353,226.706159,SCOPPITO,1,0.000000,1,0,NaN,0.418683,0.085148,0.570
1,IMA C80/2,88.910325,69.256522,75.232693,89.680834,323.080373,233.399540,SCOPPITO,2,6.395020,2,0,6.395020,0.451031,0.043365,0.580
2,L18 Packaging Line,88.847371,74.226704,89.407996,89.427519,341.909591,252.482071,Tours,3,25.224238,3,0,18.829217,0.377683,0.086173,0.547
3,L25 Packaging Line,88.849848,74.908842,90.011979,89.862601,343.633269,253.770668,Tours,4,26.947916,4,0,1.723678,0.351564,0.001613,0.478
4,M22 Filling,88.954055,81.806345,84.857670,89.522488,345.140559,255.618071,Frankfurt,5,28.455206,5,0,1.507290,0.530068,0.120280,0.650
5,M21 Filling,89.248972,83.742471,84.767181,89.760825,347.519450,257.758625,Frankfurt,6,30.834097,6,0,2.378891,0.599671,0.022006,0.650
6,M18 Filling,89.960538,82.596174,85.446232,90.979137,348.982081,258.002944,Frankfurt,7,32.296728,7,0,1.462632,0.443522,-0.010057,0.650
7,AL6,89.137919,90.323044,84.205598,89.849765,353.516326,263.666561,Frankfurt,8,36.830973,8,0,4.534245,0.332657,0.078541,0.450
8,SUPPO Packaging Line,88.585451,89.604811,89.904755,89.318502,357.413519,268.095017,Lisieux,9,40.728166,10,1,3.897193,0.353021,-0.010676,0.530
9,LINE 01 - UHLMANN 1880,89.960538,88.525570,89.636847,89.547483,357.670438,268.122955,SUZANO,10,40.985085,11,1,0.256919,0.427854,0.015085,0.650


#### write this out for tableau

In [413]:
pivot.to_csv(output_dir + "leaderboard.csv")
pivot.to_excel(output_dir + "leaderboard.xlsx")

END OF PROCESSING - Sanity checks below

In [414]:
pivot.sort_values(pivot.columns[1], inplace=True)
pivot['apr_position'] = np.arange(1,len(pivot) + 1)
pivot['temp_race_time'] = pivot[pivot.columns[[1,2]]].sum(axis=1)
pivot.sort_values(by='temp_race_time', inplace=True)
pivot['may_position'] = np.arange(1,len(pivot) + 1)
pivot['temp_race_time'] = pivot[pivot.columns[[1,2,3]]].sum(axis=1)
pivot.sort_values(by='temp_race_time', inplace=True)
pivot['jun_position'] = np.arange(1,len(pivot) + 1)
pivot['temp_race_time'] = pivot[pivot.columns[[1,2,3,4]]].sum(axis=1)
pivot.sort_values(by='temp_race_time', inplace=True)
pivot['jly_position'] = np.arange(1,len(pivot) + 1)

In [415]:
# regex filter on column name contains Line, position or 2021
pivot.filter(regex='Line|position|2021')

,Line,2021-04-15 00:00:00,2021-05-13 00:00:00,2021-06-10 00:00:00,2021-07-08 00:00:00,position,prev_position,apr_position,may_position,jun_position,jly_position
0,GAMMA1,89.153635,69.325767,68.226756,89.979194,1,1,10,2,1,1
1,IMA C80/2,88.910325,69.256522,75.232693,89.680834,2,2,5,1,2,2
2,L18 Packaging Line,88.847371,74.226704,89.407996,89.427519,3,3,3,3,3,3
3,L25 Packaging Line,88.849848,74.908842,90.011979,89.862601,4,4,4,4,4,4
4,M22 Filling,88.954055,81.806345,84.857670,89.522488,5,5,7,5,5,5
5,M21 Filling,89.248972,83.742471,84.767181,89.760825,6,6,11,7,6,6
6,M18 Filling,89.960538,82.596174,85.446232,90.979137,7,7,13,6,7,7
7,AL6,89.137919,90.323044,84.205598,89.849765,8,8,9,15,8,8
8,SUPPO Packaging Line,88.585451,89.604811,89.904755,89.318502,9,10,1,8,10,9
9,LINE 01 - UHLMANN 1880,89.960538,88.525570,89.636847,89.547483,10,11,14,10,11,10


In [416]:
# df_weekly.groupby(['Review_Date','Line']).lap_time.sum().reset_index()
sectors = df_weekly.filter(regex='Line|sector|Review_Date')

In [417]:
# sectors[sectors.columns[[2,3,4,5]]].sum(axis=1)
sectors = df_weekly.filter(regex='Line|sector|Review_Date')
sectors = sectors.iloc[:,0:6]
sectors = sectors.groupby(['Line','Review_Date']).sum().reset_index()
sectors['time'] = sectors.sum(axis=1)
sectors_pivot = sectors.pivot(index='Line', columns='Review_Date', values='time')
sectors_pivot.reset_index(inplace=True)
sectors_pivot
# sectors[sectors[sectors.columns[[2,3,4,5]]].sum(axis=1)
# sectors[sectors.Line.str.contains('AL5')]

Review_Date,Line,2021-04-15 00:00:00,2021-05-13 00:00:00,2021-06-10 00:00:00,2021-07-08 00:00:00
0,AL5 Packaging 1,1.248972,1.725754,2.495004,1.559946
1,AL6,1.137919,2.323044,1.805598,1.849765
2,C2 Packaging Line,0.952755,2.451874,2.261138,2.594185
3,C9 Packaging Line,1.021344,1.747841,2.361488,2.130804
4,GAMMA1,1.153635,1.725767,1.926756,1.979194
5,IMA C80/2,0.910325,1.656522,1.732693,1.680834
6,L18 Packaging Line,0.847371,1.426704,1.407996,1.427519
7,L25 Packaging Line,0.849848,2.108842,2.011979,1.862601
8,LINE 01 - UHLMANN 1880,NaN,0.525570,1.636847,1.547483
9,M18 Filling,1.960538,2.596174,2.646232,2.979137


In [418]:
# search for cols [1:] and find all cols after Date and Line
sectors_pivot.iloc[:,1:] = sectors_pivot.iloc[:,1:].fillna(sectors_pivot.iloc[:,1:].max())


In [419]:
sectors_pivot.sort_values(sectors_pivot.columns[1], inplace=True)
sectors_pivot['apr_position'] = np.arange(1,len(sectors_pivot) + 1)
sectors_pivot['temp_race_time'] = sectors_pivot[sectors_pivot.columns[[1,2]]].sum(axis=1)
sectors_pivot.sort_values(by='temp_race_time', inplace=True)
sectors_pivot['may_position'] = np.arange(1,len(sectors_pivot) + 1)
sectors_pivot['temp_race_time'] = sectors_pivot[sectors_pivot.columns[[1,2,3]]].sum(axis=1)
sectors_pivot.sort_values(by='temp_race_time', inplace=True)
sectors_pivot['jun_position'] = np.arange(1,len(sectors_pivot) + 1)
sectors_pivot['temp_race_time'] = sectors_pivot[sectors_pivot.columns[[1,2,3,4]]].sum(axis=1)
sectors_pivot.sort_values(by='temp_race_time', inplace=True)
sectors_pivot['jly_position'] = np.arange(1,len(sectors_pivot) + 1)
sectors_pivot.drop(columns='temp_race_time', inplace=True)
sectors_pivot

Review_Date,Line,2021-04-15 00:00:00,2021-05-13 00:00:00,2021-06-10 00:00:00,2021-07-08 00:00:00,apr_position,may_position,jun_position,jly_position
6,L18 Packaging Line,0.847371,1.426704,1.407996,1.427519,3,2,1,1
13,SUPPO Packaging Line,0.585451,1.604811,1.904755,1.318502,1,1,3,2
8,LINE 01 - UHLMANN 1880,1.960538,0.525570,1.636847,1.547483,13,4,4,3
5,IMA C80/2,0.910325,1.656522,1.732693,1.680834,5,5,5,4
14,TR200 Packaging Line,0.710480,1.591164,1.381941,2.469644,2,3,2,5
11,M22 Filling,0.954055,1.806345,2.057670,1.522488,7,6,7,6
4,GAMMA1,1.153635,1.725767,1.926756,1.979194,10,8,6,7
12,MEDISEAL PURAN,1.960538,1.020162,1.838256,1.971539,15,11,8,8
7,L25 Packaging Line,0.849848,2.108842,2.011979,1.862601,4,9,9,9
0,AL5 Packaging 1,1.248972,1.725754,2.495004,1.559946,11,10,12,10


In [420]:
pivot.filter(regex='Line|position|2021').sort_values('jun_position')

,Line,2021-04-15 00:00:00,2021-05-13 00:00:00,2021-06-10 00:00:00,2021-07-08 00:00:00,position,prev_position,apr_position,may_position,jun_position,jly_position
0,GAMMA1,89.153635,69.325767,68.226756,89.979194,1,1,10,2,1,1
1,IMA C80/2,88.910325,69.256522,75.232693,89.680834,2,2,5,1,2,2
2,L18 Packaging Line,88.847371,74.226704,89.407996,89.427519,3,3,3,3,3,3
3,L25 Packaging Line,88.849848,74.908842,90.011979,89.862601,4,4,4,4,4,4
4,M22 Filling,88.954055,81.806345,84.857670,89.522488,5,5,7,5,5,5
5,M21 Filling,89.248972,83.742471,84.767181,89.760825,6,6,11,7,6,6
6,M18 Filling,89.960538,82.596174,85.446232,90.979137,7,7,13,6,7,7
7,AL6,89.137919,90.323044,84.205598,89.849765,8,8,9,15,8,8
10,TR200 Packaging Line,88.710480,89.591164,89.381941,90.469644,11,9,2,9,9,11
8,SUPPO Packaging Line,88.585451,89.604811,89.904755,89.318502,9,10,1,8,10,9


In [421]:
aggregations = {
    'sector_1':'sum',
    'sector_2':'mean',
    'sector_3':'mean',
    'sector_4':'mean',
    'sector_5':'sum',
    'sector_6':'sum',
    'sector_7':'sum',
    'sector_8':'sum',
    'sector_9':'sum',
    'lap_time':'sum'
}
all_sectors = df_weekly.groupby(['Review_Date','Line'])[['sector_1','sector_2','sector_3','sector_4','sector_5','sector_6','sector_7','sector_8','sector_9','lap_time']].agg(aggregations).reset_index()

In [422]:
all_sectors[all_sectors.Review_Date == '2021-07-08'].sort_values('sector_1')

,Review_Date,Line,sector_1,sector_2,sector_3,sector_4,sector_5,sector_6,sector_7,sector_8,sector_9,lap_time
43,2021-07-08,AL5 Packaging 1,-0.469051,0.191344,0.241631,0.074274,0.0,0.0,0.0,0.0,0.0,1.559946
56,2021-07-08,SUPPO Packaging Line,-0.172830,0.072115,0.244240,0.056478,0.0,0.0,0.0,0.0,0.0,1.318502
50,2021-07-08,L25 Packaging Line,-0.025216,0.021244,0.198590,0.252119,0.0,0.0,0.0,0.0,0.0,1.862601
47,2021-07-08,GAMMA1,-0.008305,0.074553,0.194326,0.227996,0.0,0.0,0.0,0.0,0.0,1.979194
48,2021-07-08,IMA C80/2,0.028888,0.044093,0.178317,0.190577,0.0,0.0,0.0,0.0,0.0,1.680834
51,2021-07-08,LINE 01 - UHLMANN 1880,0.030771,0.105944,0.210129,0.063105,0.0,0.0,0.0,0.0,0.0,1.547483
54,2021-07-08,M22 Filling,0.043290,0.112922,0.127066,0.129812,0.0,0.0,0.0,0.0,0.0,1.522488
53,2021-07-08,M21 Filling,0.054741,0.156167,0.137801,0.132553,0.0,0.0,0.0,0.0,0.0,1.760825
46,2021-07-08,C9 Packaging Line,0.105945,0.060942,0.380177,0.065095,0.0,0.0,0.0,0.0,0.0,2.130804
49,2021-07-08,L18 Packaging Line,0.106991,0.044049,0.114387,0.171696,0.0,0.0,0.0,0.0,0.0,1.427519


In [423]:
all_sectors['lap_time'] = all_sectors['lap_time'] + 88

In [424]:
all_sectors

,Review_Date,Line,sector_1,sector_2,sector_3,sector_4,sector_5,sector_6,sector_7,sector_8,sector_9,lap_time
0,2021-04-15,AL5 Packaging 1,0.000000,0.048858,0.429527,0.146101,0.0,0.0,0.0,0.0,0.0,89.248972
1,2021-04-15,AL6,-0.028024,0.051409,0.367537,0.164026,0.0,0.0,0.0,0.0,0.0,89.137919
2,2021-04-15,C2 Packaging Line,-0.133204,0.074417,0.429527,0.039035,0.0,0.0,0.0,0.0,0.0,88.952755
3,2021-04-15,C9 Packaging Line,0.269447,0.088730,0.225816,0.061403,0.0,0.0,0.0,0.0,0.0,89.021344
4,2021-04-15,GAMMA1,0.028993,0.063529,0.305332,0.193460,0.0,0.0,0.0,0.0,0.0,89.153635
5,2021-04-15,IMA C80/2,0.017397,0.101067,0.244322,0.101075,0.0,0.0,0.0,0.0,0.0,88.910325
6,2021-04-15,L18 Packaging Line,-0.033218,0.068311,0.236805,0.135179,0.0,0.0,0.0,0.0,0.0,88.847371
7,2021-04-15,L25 Packaging Line,0.004575,0.065387,0.208968,0.148281,0.0,0.0,0.0,0.0,0.0,88.849848
8,2021-04-15,M18 Filling,-0.019499,0.196253,0.177578,0.616187,0.0,0.0,0.0,0.0,0.0,89.960538
9,2021-04-15,M21 Filling,0.000000,0.048858,0.429527,0.146101,0.0,0.0,0.0,0.0,0.0,89.248972


In [425]:
df_weekly[['Line','OEE %','rolling_std','Unplanned_tech_loss','Changeover_rolling_mean','Review_Date','Date']][df_weekly.Line.str.contains('M18')]

,Line,OEE %,rolling_std,Unplanned_tech_loss,Changeover_rolling_mean,Review_Date,Date
9,M18 Filling,0.046124,NaN,0.114818,0.789781,2021-04-15,2021-04-04
18,M18 Filling,0.463021,0.294791,0.240339,0.442593,2021-04-15,2021-04-11
32,M18 Filling,0.386723,0.221973,0.069673,0.425201,2021-05-13,2021-04-18
47,M18 Filling,0.352420,0.183226,0.485988,0.327211,2021-05-13,2021-04-25
63,M18 Filling,0.530336,0.079604,0.311511,0.137063,2021-05-13,2021-05-02
78,M18 Filling,0.562696,0.103957,0.227898,0.122545,2021-05-13,2021-05-09
92,M18 Filling,0.698908,0.142578,0.108069,0.024941,2021-06-10,2021-05-16
98,M18 Filling,0.000000,0.307459,0.000000,0.266630,2021-06-10,2021-05-23
116,M18 Filling,0.191951,0.323388,0.080336,0.415980,2021-06-10,2021-05-30
130,M18 Filling,0.537044,0.318333,0.216673,0.416194,2021-06-10,2021-06-06


In [426]:
df_weekly[['Line','Review_Date','Changeover_rolling_mean']].groupby(['Line','Review_Date']).sum().reset_index()

,Line,Review_Date,Changeover_rolling_mean
0,AL5 Packaging 1,2021-04-15,0.000000
1,AL5 Packaging 1,2021-05-13,0.373511
2,AL5 Packaging 1,2021-06-10,0.304670
3,AL5 Packaging 1,2021-07-08,0.297097
4,AL6,2021-04-15,0.328052
5,AL6,2021-05-13,0.378869
6,AL6,2021-06-10,0.328668
7,AL6,2021-07-08,0.271816
8,C2 Packaging Line,2021-04-15,0.078069
9,C2 Packaging Line,2021-05-13,0.140236


In [427]:
all_sectors.pivot(index='Line', columns='Review_Date', values='lap_time')

Review_Date,2021-04-15,2021-05-13,2021-06-10,2021-07-08
Line,,,,
AL5 Packaging 1,89.248972,89.725754,90.495004,89.559946
AL6,89.137919,90.323044,84.205598,89.849765
C2 Packaging Line,88.952755,90.451874,90.261138,90.594185
C9 Packaging Line,89.021344,89.747841,90.361488,90.130804
GAMMA1,89.153635,69.325767,68.226756,89.979194
IMA C80/2,88.910325,69.256522,75.232693,89.680834
L18 Packaging Line,88.847371,74.226704,89.407996,89.427519
L25 Packaging Line,88.849848,74.908842,90.011979,89.862601
LINE 01 - UHLMANN 1880,NaN,88.525570,89.636847,89.547483


adding a weekly race_time just for the gapminder presentation

In [428]:
weekly_race_time = df_weekly.groupby(['Line','Date'])['lap_time'].sum().groupby('Line').cumsum().reset_index()
weekly_race_time.rename(columns={'lap_time' : 'race_time'}, inplace=True)
weekly_race_time['race_time'] = weekly_race_time['race_time'] + 88
df_weekly = df_weekly.merge(weekly_race_time[['Line','Date','race_time']], on=(['Line','Date']))
# df_weekly.drop(columns={'race_time'}, inplace=True)
df_weekly

,Week,Line,OEE %,WeekOfYear,Year,Date,Plant,OEE Start point,OEE% Target (2022),Unplanned_tech_loss,...,sector_2,sector_3,sector_4,sector_5,sector_6,sector_7,sector_8,sector_9,lap_time,race_time
0,W13-2021,C2 Packaging Line,0.458414,13.0,2021.0,2021-04-04,Maisons-Alfort,0.397503,0.470,0.465009,...,0.097716,0.465009,0.044140,0.0,0.0,0.0,0.0,0.0,0.545954,88.545954
1,W13-2021,L18 Packaging Line,0.465923,13.0,2021.0,2021-04-04,Tours,0.377683,0.547,0.208273,...,0.097716,0.208273,0.134271,0.0,0.0,0.0,0.0,0.0,0.352020,88.352020
2,W13-2021,IMA C80/2,0.581304,13.0,2021.0,2021-04-04,SCOPPITO,0.451031,0.580,0.224561,...,0.097716,0.224561,0.070836,0.0,0.0,0.0,0.0,0.0,0.262841,88.262841
3,W13-2021,AL6,0.367897,13.0,2021.0,2021-04-04,Frankfurt,0.332657,0.450,0.342851,...,0.097716,0.342851,0.184958,0.0,0.0,0.0,0.0,0.0,0.590284,88.590284
4,W13-2021,TR200 Packaging Line,0.596432,13.0,2021.0,2021-04-04,Lisieux,0.483505,0.650,0.163573,...,0.097716,0.163573,0.075189,0.0,0.0,0.0,0.0,0.0,0.223551,88.223551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,W26-2021,GAMMA1,0.491885,26.0,2021.0,2021-07-04,SCOPPITO,0.418683,0.570,0.143339,...,0.055086,0.143339,0.219199,-0.0,-0.0,-0.0,-0.0,-0.0,0.528796,52.685353
198,W26-2021,C9 Packaging Line,0.406623,26.0,2021.0,2021-07-04,Maisons-Alfort,0.528518,0.530,0.452948,...,0.081720,0.452948,0.064467,-0.0,-0.0,-0.0,-0.0,-0.0,0.681253,95.261477
199,W26-2021,L18 Packaging Line,0.408843,26.0,2021.0,2021-07-04,Tours,0.377683,0.547,0.184455,...,0.054692,0.184455,0.182758,-0.0,-0.0,-0.0,-0.0,-0.0,0.506118,77.909591
200,W26-2021,L25 Packaging Line,0.381273,26.0,2021.0,2021-07-04,Tours,0.351564,0.478,0.192098,...,0.019916,0.192098,0.252406,-0.0,-0.0,-0.0,-0.0,-0.0,0.427800,79.633269


In [429]:
all_sectors_pivot.to_excel(dir +'all_sectors_pivot.xlsx')

In [430]:
gapminder_weekly = df_weekly[['Line','Date','sector_1','sector_2','sector_3','sector_4','lap_time','race_time']]
gapminder_weekly.to_excel(dir +'gapfinder_weekly.xlsx', index=False)

In [432]:
df_weekly[['Line','Date','race_time']][df_weekly.Line.str.contains('TR200')].sort_values(['Line','Date'])

,Line,Date,race_time
4,TR200 Packaging Line,2021-04-04,88.223551
15,TR200 Packaging Line,2021-04-11,88.710480
26,TR200 Packaging Line,2021-04-18,88.977055
48,TR200 Packaging Line,2021-04-25,89.573539
55,TR200 Packaging Line,2021-05-02,90.001985
69,TR200 Packaging Line,2021-05-09,90.301644
96,TR200 Packaging Line,2021-05-16,90.694533
106,TR200 Packaging Line,2021-05-23,90.993509
118,TR200 Packaging Line,2021-05-30,91.521049
137,TR200 Packaging Line,2021-06-06,91.683586
